In [ ]:
!pip install datasets 
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import TFAutoModelForTokenClassification, TrainingArguments, Trainer, AutoTokenizer
from datasets import Dataset
import tensorflow as tf

gmb = pd.read_csv("/content/gmb.csv", sep=",", encoding='unicode_escape').head(100_000).dropna()


X = gmb.Word.astype(str).tolist()
y_ner = gmb.Tag.tolist()


X_train, X_test, y_train, y_test = train_test_split(X, y_ner, test_size=0.2, random_state=404)


train_dataset = Dataset.from_dict({"text": X_train, "labels": y_train})
test_dataset = Dataset.from_dict({"text": X_test, "labels": y_test})

tokenizer = AutoTokenizer.from_pretrained("dslim/distilbert-NER")
model = TFAutoModelForTokenClassification.from_pretrained("dslim/distilbert-NER", num_labels=len(set(y_ner)))


training_args = TrainingArguments(
    output_dir='./results',          
    evaluation_strategy="epoch",    
    save_strategy="epoch",           
    learning_rate=2e-5,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=16,   
    num_train_epochs=3,              
    weight_decay=0.01,               
    logging_dir='./logs',           
    logging_steps=200,               
    load_best_model_at_end=True,     
)



trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,        
    eval_dataset=test_dataset,          
    compute_metrics=compute_metrics      
)


trainer.train()